#    Music Generation by Restricted Boltzmann Machine (RBM)

In [1]:
#importing all the python libraries that we need for the Music Generation

In [2]:
import numpy as np
import pandas as pd
import msgpack
import glob
import tensorflow as tf
from tensorflow.python.ops import control_flow_ops
from tqdm import tqdm
import midi_manipulation

C:\Users\cheta\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
# Import the Dataset to the network to train the machine

In [4]:
def get_songs(path):
    files = glob.glob('{}/*.mid*'.format(path))
    songs = []
    for f in tqdm(files):
        try:
            song = np.array(midi_manipulation.midiToNoteStateMatrix(f))
            if np.array(song).shape[0] > 50:
                songs.append(song)
        except Exception as e:
            raise e           
    return songs

In [5]:
songs = get_songs('Pop_Music_Midi') #These songs have already been converted from midi to msgpack
print ("{} songs processed".format(len(songs)))

100%|████████████████████████████████████████████████████████████████████████████████| 126/126 [00:03<00:00, 40.46it/s]


122 songs processed


In [6]:
# HyperParameters

# The hyperparameters of our models are initilised as below

In [7]:
# This gives us the lowest note on the piano roll
lowest_note = midi_manipulation.lowerBound
print(lowest_note)

24


In [8]:
# This gives us the highest note on the piano roll
highest_note = midi_manipulation.upperBound
print(highest_note)

102


In [9]:
# The note range of the piano roll will be given by the difference between upperBound and the lowerBound
note_range = highest_note - lowest_note
print(note_range)

78


In [10]:
#num_timesteps is the number of timesteps that we will create at a time
num_timesteps  = 15

In [11]:
# Initialising the size of visible layer
n_visible      = 2 * note_range * num_timesteps
print(n_visible)

2340


In [12]:
# Initialising the size of hidden layer
n_hidden = 50

In [13]:
#The number of training epochs that we are going to run. For each epoch we go through the entire data set.
num_epochs = 200

In [14]:
#The number of training examples that we are going to send through the RBM at a time. 
batch_size = 100

In [15]:
#The learning rate of our model
lr = tf.constant(0.005, tf.float32)
print(lr)

Tensor("Const:0", shape=(), dtype=float32)


In [16]:
# Variables that we are going to use for the model
# lets initialize the variables

In [17]:
# x is the placeholder variable that holds our data
x  = tf.placeholder(tf.float32, [None, n_visible], name="x")
print(x)

Tensor("x:0", shape=(?, 2340), dtype=float32)


In [18]:
#W is the weight matrix that stores the edge weights
W  = tf.Variable(tf.random_normal([n_visible, n_hidden], 0.01), name="W")
print(W)

<tf.Variable 'W:0' shape=(2340, 50) dtype=float32_ref>


In [19]:
#bh is the bias vector for the hidden layer
bh = tf.Variable(tf.zeros([1, n_hidden],  tf.float32, name="bh"))
print(bh)

<tf.Variable 'Variable:0' shape=(1, 50) dtype=float32_ref>


In [20]:
#bv is the bias vector for the visible layer
bv = tf.Variable(tf.zeros([1, n_visible],  tf.float32, name="bv"))
print(bv)

<tf.Variable 'Variable_1:0' shape=(1, 2340) dtype=float32_ref>


In [21]:
# Helper functions

In [22]:
#sample function lets us easily sample from a vector of probabilities
#Takes in a vector of probabilities, and returns a random vector of 0s and 1s sampled from the input vector
def sample(probs):
    return tf.floor(probs + tf.random_uniform(tf.shape(probs), 0, 1))

In [23]:
#This function runs the gibbs chain. We will call this function in two places:
#    - When we define the training update step
#    - When we sample our music segments from the trained RBM
def gibbs_sample(k):
    #Runs a k-step gibbs chain to sample from the probability distribution of the RBM defined by W, bh, bv
    def gibbs_step(count, k, xk):
        #Runs a single gibbs step. The visible values are initialized to xk
        hk = sample(tf.sigmoid(tf.matmul(xk, W) + bh)) #Propagate the visible values to sample the hidden values
        xk = sample(tf.sigmoid(tf.matmul(hk, tf.transpose(W)) + bv)) #Propagate the hidden values to sample the visible values
        return count+1, k, xk
    #Run gibbs steps for k iterations
    ct = tf.constant(0) #counter
    [_, _, x_sample] = control_flow_ops.while_loop(lambda count, num_iter, *args: count < num_iter,
                                         gibbs_step, [ct, tf.constant(k), x])
    #This is not strictly necessary in this implementation, but if you want to adapt this code to use one of TensorFlow's
    #optimizers, you need this in order to stop tensorflow from propagating gradients back through the gibbs step
    x_sample = tf.stop_gradient(x_sample) 
    return x_sample

In [24]:
### Training Update Code
# Now we implement the contrastive divergence algorithm. First, we get the samples of x and h from the probability distribution


In [25]:
#The sample of x
x_sample = gibbs_sample(1)
print(x_sample)

Tensor("StopGradient:0", shape=(?, 2340), dtype=float32)


In [26]:
#The sample of the hidden nodes, starting from the visible state of x
h = sample(tf.sigmoid(tf.matmul(x, W) + bh))
print(h)

Tensor("Floor:0", shape=(?, 50), dtype=float32)


In [28]:
#The sample of the hidden nodes, starting from the visible state of x_sample
h_sample = sample(tf.sigmoid(tf.matmul(x_sample, W) + bh)) 
print(h_sample)

Tensor("Floor_1:0", shape=(?, 50), dtype=float32)


In [29]:
#now we update the values of W, bh, and bv, based on the difference between the samples that we drew and the original values


In [30]:
size_bt = tf.cast(tf.shape(x)[0], tf.float32)
print(size_bt)

Tensor("Cast:0", shape=(), dtype=float32)


In [31]:
W_adder  = tf.multiply(lr/size_bt, tf.subtract(tf.matmul(tf.transpose(x), h), tf.matmul(tf.transpose(x_sample), h_sample)))
print(W_adder)

Tensor("Mul:0", shape=(2340, 50), dtype=float32)


In [32]:
bv_adder = tf.multiply(lr/size_bt, tf.reduce_sum(tf.subtract(x, x_sample), 0, True))
print(bv_adder)

Tensor("Mul_1:0", shape=(1, 2340), dtype=float32)


In [33]:
bh_adder = tf.multiply(lr/size_bt, tf.reduce_sum(tf.subtract(h, h_sample), 0, True))
print(bh_adder)

Tensor("Mul_2:0", shape=(1, 50), dtype=float32)


In [34]:
#When we do sess.run(updt), TensorFlow will run all 3 update steps
updt = [W.assign_add(W_adder), bv.assign_add(bv_adder), bh.assign_add(bh_adder)]
print(updt)

[<tf.Tensor 'AssignAdd:0' shape=(2340, 50) dtype=float32_ref>, <tf.Tensor 'AssignAdd_1:0' shape=(1, 2340) dtype=float32_ref>, <tf.Tensor 'AssignAdd_2:0' shape=(1, 50) dtype=float32_ref>]


In [35]:
# Running the tensorflow session

In [36]:
with tf.Session() as sess:
    #First, we train the model
    #initialize the variables of the model
    init = tf.global_variables_initializer()
    sess.run(init)
    
    #Run through all of the training data num_epochs times
    
    for epoch in tqdm(range(num_epochs)):
        for song in songs:
            
            #The songs are stored in a time x notes format. The size of each song is timesteps_in_song x 2*note_range
            #Here we reshape the songs so that each training example is a vector with num_timesteps x 2*note_range elements
            
            song = np.array(song)
            song = song[:int(np.floor(song.shape[0]//num_timesteps)*num_timesteps)]
            song = np.reshape(song, [song.shape[0]//num_timesteps, song.shape[1]*num_timesteps])
            
            #Train the RBM on batch_size examples at a time
            
            for i in range(1, len(song), batch_size): 
                tr_x = song[i:i+batch_size]
                sess.run(updt, feed_dict={x: tr_x})

    #Now the model is fully trained, so let's make some music! 
    #Run a gibbs chain where the visible nodes are initialized to 0
    
    sample = gibbs_sample(1).eval(session=sess, feed_dict={x: np.zeros((200, n_visible))})
    for i in range(sample.shape[0]):
        if not any(sample[i,:]):
            continue
            
        #Here we reshape the vector to be time x notes, and then save the vector as a midi file
        
        Music = np.reshape(sample[i,:], (num_timesteps, 2*note_range))
        midi_manipulation.noteStateMatrixToMidi(Music, "Output//generated_chord_Music_{}".format(i))

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [00:44<00:00,  4.46it/s]


** the generated chord music is now placed in the output folder but these are in small snippets. So we merge these files to one big music file which finalises the MUSIC GENERATION**

In [37]:
# All the libraries which are needed to merge the generated chords to one final music midi file are already imported to this notebook

In [38]:
try:
    files = glob.glob('Output//generated*.mid*')
except Exception as e:
    raise e

In [39]:
print(files)

['Output\\generated_chord_Music_0.mid', 'Output\\generated_chord_Music_1.mid', 'Output\\generated_chord_Music_10.mid', 'Output\\generated_chord_Music_100.mid', 'Output\\generated_chord_Music_101.mid', 'Output\\generated_chord_Music_102.mid', 'Output\\generated_chord_Music_103.mid', 'Output\\generated_chord_Music_104.mid', 'Output\\generated_chord_Music_105.mid', 'Output\\generated_chord_Music_106.mid', 'Output\\generated_chord_Music_107.mid', 'Output\\generated_chord_Music_108.mid', 'Output\\generated_chord_Music_109.mid', 'Output\\generated_chord_Music_11.mid', 'Output\\generated_chord_Music_110.mid', 'Output\\generated_chord_Music_111.mid', 'Output\\generated_chord_Music_112.mid', 'Output\\generated_chord_Music_113.mid', 'Output\\generated_chord_Music_114.mid', 'Output\\generated_chord_Music_115.mid', 'Output\\generated_chord_Music_116.mid', 'Output\\generated_chord_Music_117.mid', 'Output\\generated_chord_Music_118.mid', 'Output\\generated_chord_Music_119.mid', 'Output\\generated_ch

In [40]:
songs = np.zeros((0,156))

In [41]:
# Merging the generated chord using numpy's concatenation method

In [42]:
for f in tqdm(files):
    try:
        song = np.array(midi_manipulation.midiToNoteStateMatrix(f))

        if np.array(song).shape[0] < 5:
            songs = np.concatenate((songs,song))
    except Exception as e:
        raise e

100%|███████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 798.86it/s]


In [43]:
print ("samlpes merging ...")
print (np.shape(songs))

samlpes merging ...
(200, 156)


In [44]:
midi_manipulation.noteStateMatrixToMidi(songs, "Output//Final_Generated_Midi")